# Visualising GPX data in Python

First we need to load all the packages, import the data and covert it into a pandas DataFrame.

In [3]:
import gpxpy
import numpy as np
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import seawater as sw


# Import data
gpx = gpxpy.parse(open('/home/sh16450/Downloads/Getting_attacked_by_British_drivers.gpx'))

print("{} track(s)".format(len(gpx.tracks)))
track = gpx.tracks[0]
print("{} segment(s)".format(len(track.segments)))
segment = track.segments[0]
print("{} point(s)".format(len(segment.points)))

data = []
segment_length = segment.length_3d()
for point_idx, point in enumerate(segment.points):
    data.append([point.longitude, point.latitude,
                 point.elevation, point.time, segment.get_speed(point_idx)])

columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
df = pd.DataFrame(data, columns=columns)
df.head()

1 track(s)
1 segment(s)
9876 point(s)


,Longitude,Latitude,Altitude,Time,Speed
0,-3.517262,50.721712,45.2,2018-01-28 10:19:56,NaN
1,-3.517270,50.721741,45.2,2018-01-28 10:19:58,1.276335
2,-3.517300,50.721714,45.2,2018-01-28 10:20:02,0.613369
3,-3.517344,50.721714,45.3,2018-01-28 10:20:12,0.257696
4,-3.517361,50.721669,45.4,2018-01-28 10:20:37,0.947626


## Smoothing

Because GPX data from a smartphone can be a bit noisy, it is better to smooth the data, e.g. with a rolling mean.

In [11]:
# SMOOTHING
df_new = df[['Longitude', 'Latitude', 'Altitude', 'Speed']].rolling(window=5, min_periods=1, center=True).mean()
df_new['Time'] = df['Time']
# smoothed total of climbs
alt_tot = df_new['Altitude'].diff()[df_new['Altitude'].diff() > 0.0].sum()
print("We've climbed {} m today!".format(round(alt_tot)))

We've climbed 611.0 m today!


## Visualising the data

After extaction and smoothing, it's time to plot the data using the mplleaflet package on a nice, zoomable map.

In [14]:
# Visualize the data
fig, ax = plt.subplots(figsize=(16,9))
df = df.dropna()
ax.plot(df_new['Longitude'], df_new['Latitude'],
        color='darkorange', linewidth=6, alpha=0.5)
mplleaflet.display(fig=fig, tiles='esri_aerial')